## Prueba de modelos ##

### Version 4: Modelo con 2 DER, planificación anual, granularidad horaria ###

Si el modelo tiene que optimizar el balance por horas, tiene 8640 VD, pero sigue teniendo 2 VD binarias

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandapower as pp

In [4]:
import os

In [5]:
import pandas as pd

In [6]:
import pyomo.environ as pe

In [7]:
import math

In [8]:
import random

In [9]:
import itertools

In [10]:
import matplotlib.pyplot as plt

In [11]:
import numpy as np

In [12]:
import ModelWriters.SimpleBusbar

Variables que define el modelo:

y para el año

d para el día del año

h para la hora del día

dt la granularidad del modelo, en horas


temp la temperatura

wv la velocidad del viento

I la irradiación solar


eg el crecimiento económico en pu respecto al año 0


Estas variables llegan como parámetros en forma de diccionario en el argumento 'model_status' (ver este nombre)

Todas las funciones reciben este argumento, la lógica de la función indica que valor retorna. Pr ejemplo, si la granularidad es 24 h, debe retornar el valor medio del parámetro producido.

In [22]:
def oc_1_ext_grid(model_status={}):
    #modelo sencillo con dos precios, uno entre 0 a 18 y otro de 18 a 24
    res = 0.0
    sx = 1e-6
    if 'h' in model_status:
        h = model_status['h']
        if 0.0 <= h and h < 18.0:
            res = 3600.0*sx
        elif 18 <= h and h < 24.0:
            res = 5400.0*sx
        else:
            raise ValueError("Hour outside model range")
    else:
        raise ValueError("Hour not defined")
    
    return res

In [28]:
def demand(model_status={}):
    #modelo sencillo en forma de escalones
    #devuelvr la fracción de la carga empleada
    #se considera que la carga tiene una variabilidad aleatoria de un 20% y es mayor en verano e invierno en un 30%
    #la variabilidad total puede ser 1.3*1.2=1.56 verano-invierno o 0.7*0.8 = 0.56 otoño-primavera, 
    res = 0.0
    if 'h' in model_status:
        h = model_status['h']
        if 0.0 <= h and h < 6.0:
            res = 0.2
        elif 6.0 <= h and h < 8.0:
            res = 0.4
        elif 8.0 <= h and h < 18.0:
            res = 0.5
        elif 18.0 <= h and h < 22.0:
            res = 1.0
        elif 22.0 <= h and h < 24.0:
            res = 0.3            
        else:
            raise ValueError("Hour outside model range")
    else:
        raise ValueError("Hour not defined")
    
    #estacionalidad
    if 'd' in model_status:
        d = model_status['d']
        #se asume que el día de mayor consumo es el inicio del verano y del invierno (21/12 y 21/07), por eso sumo 10 días el dia 0
        #divido por 180 porque hay dos picos en el año y paso a radianes
        estacionalidad = 1 + 0.3*math.cos((d + 10.0)/180.0*2*math.pi)  
    else:
        raise ValueError("Day not defined")
    res = res * estacionalidad
    
    #Aleatoriedad:
    res = res*random.uniform(0.8, 1.2)
    return res

In [31]:
def solar_output(model_status={}):
    #modelo sencillo
    #usando los datos de radiacion solar entregada por el modelo
    #considero el valor normal 1000 W/m2
    res = 0.0
    if 'I' in model_status:
        I = model_status['I']
        if 0.0 <= I and I < 1200:
            res = I / 1000.0
        else:
            raise ValueError("Solar radiation outside model range")
    else:
        raise ValueError("Solar radiation not defined")
    
    return res

In [34]:
def wind_output(model_status={}):
    #modelo sencillo
    #devuelvr la fracción de la potencia entregada en función del tiempo
    #no toma en cuenta la radicación solar ni la temperatura
    res = 0.0
    if 'wv' in model_status:
        wv = model_status['wv']
        if 0.0 <= wv and wv < 3.0:
            res = 0.0
        elif 3.0 <= wv and wv < 15.0:
            res = (wv-3.0)/(15.0-3.0)
        elif 15.0 <= wv and wv < 25.0:
            res = 1.0
        else:
            res = 0.0
    else:
        raise ValueError("Wind Velocity not defined")
    
    return res

## Construcción de Escenarios

In [79]:
dias = 3
años = 5

In [80]:
#el rango de horas:
T_i = range(24)

#el rango de dias:
Dias_i = range(dias)

Escenarios_i = range(len(T_i)*len(Dias_i))

In [81]:
def solar_irradiance_simulation(d, h):
    #simula las horas de sol, la intensidad y los dias multiplos de 5 esta nublado, con un 20% de la radiacion
    #a todo esto se le aplica una aleatoriedad del 80 al 100%
    #los coeficientes son aproximados para Buenos Aires
    radiacion = 1000.0
    
    estacionalidad = 0.75 + 0.25*math.cos((d + 10.0)/365.0*2*math.pi)  
    
    if d % 5 == 0 :
        nubosidad = 0.2
    else:
        nubosidad = 1.0
        
    aleatoriedad = random.uniform(0.8, 1.0)
    
    salida = 7.0 - 1.1*math.cos((d + 10.0)/365.0*2*math.pi)    #es una aproximacion, no basada en modelos matematicos
    puesta = 19.0 + 1.1*math.cos((d + 10.0)/365.0*2*math.pi)
    
    temporalidad = 0.0
    if salida <= h and h <= puesta:
        temporalidad = math.sin((h-salida)/(puesta-salida)*math.pi)
        
    return radiacion*estacionalidad*nubosidad*aleatoriedad*temporalidad

In [82]:
m_s_base = [{'y': 0, 'd': 0, 'dd':1.0, 'h': 0.0, 'dt': 1.0, 'temp': 20.0, 'I':1000.0, 'wv': 10.0, 'eg': 1.0}]

registros = len(Dias_i)*len(T_i)

Escenarios =  pd.DataFrame(m_s_base, index = Escenarios_i)

reg = 0
for d in Dias_i:
    for h in T_i:
        Escenarios['d'][reg] = d
        Escenarios['h'][reg] = h
        Escenarios['I'][reg] = solar_irradiance_simulation(d, h)
        Escenarios['wv'][reg] = random.uniform(1.0, 20.0)
        reg += 1

In [83]:
Escenarios

,y,d,dd,h,dt,temp,I,wv,eg
0,0,0,1.0,0.0,1.0,20.0,0.000000,8.809258,1.0
1,0,0,1.0,1.0,1.0,20.0,0.000000,14.994982,1.0
2,0,0,1.0,2.0,1.0,20.0,0.000000,11.665953,1.0
3,0,0,1.0,3.0,1.0,20.0,0.000000,5.649301,1.0
4,0,0,1.0,4.0,1.0,20.0,0.000000,15.730802,1.0
...,...,...,...,...,...,...,...,...,...
67,0,2,1.0,19.0,1.0,20.0,225.165282,8.822528,1.0
68,0,2,1.0,20.0,1.0,20.0,16.655944,4.532359,1.0
69,0,2,1.0,21.0,1.0,20.0,0.000000,4.396741,1.0
70,0,2,1.0,22.0,1.0,20.0,0.000000,9.416092,1.0


## Model Tests

In [228]:
from ModelWriters.Resources.ExtGrid import ExtGrid

In [229]:
from ModelWriters.Resources.Load import Load

In [230]:
from ModelWriters.Resources.BaseGen import Generator

In [231]:
model = pe.ConcreteModel()

In [232]:
exg = ExtGrid('EXT')

In [233]:
bg = Generator('GEN')

In [236]:
load = Load('LOAD')

In [220]:
exg['pr_mw'] = 5.0
exg['ic_0_mu'] = 1.2
exg['ic_1_mu'] = 0.2
exg['oc_0_mu'] = 0.001
exg['oc_1_mu'] = 0.005

#exg.pa_pu = wind_output

In [248]:
load['pr_mw'] = .5
load['ic_0_mu'] = 1.2
load['ic_1_mu'] = 0.2
load['oc_0_mu'] = 0.001
load['oc_1_mu'] = 0.005
load['pa_pu'] = demand

#exg.pa_pu = wind_output

In [188]:
exg.initialize_model(model, Escenarios)

In [221]:
bg['pr_mw'] = 5.0
bg['ic_0_mu'] = 1.2
bg['ic_1_mu'] = 0.2
bg['oc_0_mu'] = 0.001
bg['oc_1_mu'] = 0.005


In [222]:
bg.initialize_model(model, Escenarios)

In [249]:
load.initialize_model(model, Escenarios)

In [251]:
load.available_power(10)

0.26805540019437046

In [223]:
bg.available_power(15)

In [227]:
print(bg.initial_cost())

1.2 + 0.2*GEN_pr_mw


In [252]:
print(load.initial_cost())

1.3


In [225]:
print(bg.operating_cost(10))

0.001 + 0.005*GEN_p_mw[10]


In [254]:
print(load.operating_cost(10))

0.0015


In [226]:
print(bg.active_power(10))

GEN_p_mw[10]


In [255]:
print(load.active_power(10))

-0.35245898398589703


In [256]:
model.pprint()

0 Declarations: 
